In [2]:
import pandas as pd
import numpy as np

In [35]:
#df_ADNR_A01 = pd.read_csv('../data_ADNR/with0.1anomaly/allocatedJobs.csv')
#df_ADNR_M01 = pd.read_csv('../data_ADNR/with0.1anomaly/destroyedJobs.csv')

df_ADNR_A01 = pd.read_csv('../data_ADNR/with0.1anomaly/allocatedJobs.csv')
df_ADNR_A01_2 = pd.read_csv('../experiments/experiment_results_NR/0.1/allocatedJobs.csv')
df_ADNR_A01 = pd.concat([df_ADNR_A01,df_ADNR_A01_2], ignore_index=True)

df_ADNR_M01 = pd.read_csv('../data_ADNR/with0.1anomaly/destroyedJobs.csv')
df_ADNR_M01_2 = pd.read_csv('../experiments/experiment_results_NR/0.1/destroyedJobs.csv')
df_ADNR_M01 = pd.concat([df_ADNR_M01,df_ADNR_M01_2], ignore_index=True)

In [4]:
def has_high_resource(row):
    # Check if any resource requirement is above 10
    return np.any(np.array(row['job ressource requirement']) > 10)

In [64]:
def convert_to_list(row):
    if isinstance(row, list):
        return row
    # Remove brackets and split on space
    items = row.replace('[', '').replace(']', '').split(' ')
    # Remove any empty strings caused by extra spaces
    items = [item for item in items if item != '']
    # Convert strings to integers
    items = [int(item) for item in items]
    return items

In [6]:
def has_high_length(row):
    return row['job Length'] >= 23

In [ ]:
df_ADNR_A01['job ressource requirement'] = df_ADNR_A01['job ressource requirement'].apply(convert_to_list)
df_ADNR_M01['job ressource requirement'] = df_ADNR_M01['job ressource requirement'].apply(convert_to_list)

df_ADNR_A01['high_resource'] = df_ADNR_A01.apply(has_high_resource, axis=1)
df_ADNR_M01['high_resource'] = df_ADNR_M01.apply(has_high_resource, axis=1)

df_ADNR_A01['high_length'] = df_ADNR_A01.apply(has_high_length, axis=1)
df_ADNR_M01['high_length'] = df_ADNR_M01.apply(has_high_length, axis=1)

df_ADNR_A01['high_length_or_resource'] = df_ADNR_A01['high_length'] | df_ADNR_A01['high_resource']
df_ADNR_M01['high_length_or_resource'] = df_ADNR_M01['high_length'] | df_ADNR_M01['high_resource']

allocated_high_length_or_resource = df_ADNR_A01['high_length_or_resource'].sum()
destroyed_high_length_or_resource = df_ADNR_M01['high_length_or_resource'].sum()

# Calculate the total number of jobs that satisfy either condition
total_high_length_or_resource = allocated_high_length_or_resource + destroyed_high_length_or_resource

# Calculate the proportion of these jobs that were destroyed
proportion_destroyed_length_or_resource = float(destroyed_high_length_or_resource) / total_high_length_or_resource

TN = len(df_ADNR_A01[(df_ADNR_A01['high_resource'] == False) & (df_ADNR_A01['high_length'] == False)])

# False negatives (anomalous jobs incorrectly identified as non-anomalies)
FN = len(df_ADNR_A01[(df_ADNR_A01['high_resource'] == True) | (df_ADNR_A01['high_length'] == True)])

TP = len(df_ADNR_M01[(df_ADNR_M01['high_resource'] == True) | (df_ADNR_M01['high_length'] == True)])

# False positives (non-anomalous jobs incorrectly identified as anomalies)
FP = len(df_ADNR_M01[(df_ADNR_M01['high_resource'] == False) & (df_ADNR_M01['high_length'] == False)])

# Recall calculation
recall = float(TP) / (TP + FN)
recall



# Precision calculation
precision = float(TP) / (TP + FP)
precision

f1_score = 2 * (precision * recall) / (precision+recall)
f1_score

print(proportion_destroyed_length_or_resource, precision, recall, f1_score)

In [36]:
df_ADNR_A01['job ressource requirement'] = df_ADNR_A01['job ressource requirement'].apply(convert_to_list)
df_ADNR_M01['job ressource requirement'] = df_ADNR_M01['job ressource requirement'].apply(convert_to_list)

In [37]:
df_ADNR_A01['high_resource'] = df_ADNR_A01.apply(has_high_resource, axis=1)
df_ADNR_M01['high_resource'] = df_ADNR_M01.apply(has_high_resource, axis=1)

In [38]:
# Apply the function to both dataframes
df_ADNR_A01['high_length'] = df_ADNR_A01.apply(has_high_length, axis=1)
df_ADNR_M01['high_length'] = df_ADNR_M01.apply(has_high_length, axis=1)

In [39]:
# Create a new column that is True when either high_length or high_resource is True
df_ADNR_A01['high_length_or_resource'] = df_ADNR_A01['high_length'] | df_ADNR_A01['high_resource']
df_ADNR_M01['high_length_or_resource'] = df_ADNR_M01['high_length'] | df_ADNR_M01['high_resource']

In [40]:
allocated_high_length_or_resource = df_ADNR_A01['high_length_or_resource'].sum()
destroyed_high_length_or_resource = df_ADNR_M01['high_length_or_resource'].sum()

# Calculate the total number of jobs that satisfy either condition
total_high_length_or_resource = allocated_high_length_or_resource + destroyed_high_length_or_resource

# Calculate the proportion of these jobs that were destroyed
proportion_destroyed_length_or_resource = float(destroyed_high_length_or_resource) / total_high_length_or_resource

print("Proportion of jobs with either high length or high resource that were destroyed:", proportion_destroyed_length_or_resource)

('Proportion of jobs with either high length or high resource that were destroyed:', 0.7476025704399407)


In [12]:
TP = len(df_ADNR_M01[(df_ADNR_M01['high_resource'] == True) | (df_ADNR_M01['high_length'] == True)])

# False positives (non-anomalous jobs incorrectly identified as anomalies)
FP = len(df_ADNR_M01[(df_ADNR_M01['high_resource'] == False) & (df_ADNR_M01['high_length'] == False)])

# Precision calculation
precision = float(TP) / (TP + FP)
precision

0.31045380875202594

In [14]:
f1_score = 2 * (precision * proportion_destroyed_length_or_resource) / (precision+proportion_destroyed_length_or_resource)
f1_score

0.43850512218851945

In [41]:
#df_ADNR_A03 = pd.read_csv('../data_ADNR/with0.3anomaly/allocatedJobs.csv')
#df_ADNR_M03 = pd.read_csv('../data_ADNR/with0.3anomaly/destroyedJobs.csv')
df_ADNR_A03 = pd.read_csv('../data_ADNR/with0.3anomaly/allocatedJobs.csv')
df_ADNR_A03_2 = pd.read_csv('../experiments/experiment_results_NR/0.3/allocatedJobs.csv')
df_ADNR_A03 = pd.concat([df_ADNR_A03,df_ADNR_A03_2], ignore_index=True)

df_ADNR_M03 = pd.read_csv('../data_ADNR/with0.3anomaly/destroyedJobs.csv')
df_ADNR_M03_2 = pd.read_csv('../experiments/experiment_results_NR/0.3/destroyedJobs.csv')
df_ADNR_M03 = pd.concat([df_ADNR_M03,df_ADNR_M03_2], ignore_index=True)


In [42]:
df_ADNR_A03['job ressource requirement'] = df_ADNR_A03['job ressource requirement'].apply(convert_to_list)
df_ADNR_M03['job ressource requirement'] = df_ADNR_M03['job ressource requirement'].apply(convert_to_list)

df_ADNR_A03['high_resource'] = df_ADNR_A03.apply(has_high_resource, axis=1)
df_ADNR_M03['high_resource'] = df_ADNR_M03.apply(has_high_resource, axis=1)

df_ADNR_A03['high_length'] = df_ADNR_A03.apply(has_high_length, axis=1)
df_ADNR_M03['high_length'] = df_ADNR_M03.apply(has_high_length, axis=1)

df_ADNR_A03['high_length_or_resource'] = df_ADNR_A03['high_length'] | df_ADNR_A03['high_resource']
df_ADNR_M03['high_length_or_resource'] = df_ADNR_M03['high_length'] | df_ADNR_M03['high_resource']

allocated_high_length_or_resource = df_ADNR_A03['high_length_or_resource'].sum()
destroyed_high_length_or_resource = df_ADNR_M03['high_length_or_resource'].sum()

# Calculate the total number of jobs that satisfy either condition
total_high_length_or_resource = allocated_high_length_or_resource + destroyed_high_length_or_resource

# Calculate the proportion of these jobs that were destroyed
proportion_destroyed_length_or_resource = float(destroyed_high_length_or_resource) / total_high_length_or_resource

TN = len(df_ADNR_A03[(df_ADNR_A03['high_resource'] == False) & (df_ADNR_A03['high_length'] == False)])

# False negatives (anomalous jobs incorrectly identified as non-anomalies)
FN = len(df_ADNR_A03[(df_ADNR_A03['high_resource'] == True) | (df_ADNR_A03['high_length'] == True)])

TP = len(df_ADNR_M03[(df_ADNR_M03['high_resource'] == True) | (df_ADNR_M03['high_length'] == True)])

# False positives (non-anomalous jobs incorrectly identified as anomalies)
FP = len(df_ADNR_M03[(df_ADNR_M03['high_resource'] == False) & (df_ADNR_M03['high_length'] == False)])

# Recall calculation
recall = float(TP) / (TP + FN)
recall



# Precision calculation
precision = float(TP) / (TP + FP)
precision

f1_score = 2 * (precision * recall) / (precision+recall)
f1_score

print(proportion_destroyed_length_or_resource, precision, recall, f1_score)

(0.8238316979876933, 0.7348612965435395, 0.8238316979876933, 0.7768072761486593)


In [43]:
#df_ADNR_A05 = pd.read_csv('../data_ADNR/with0.5anomaly/allocatedJobs.csv')
#df_ADNR_M05 = pd.read_csv('../data_ADNR/with0.5anomaly/destroyedJobs.csv')

df_ADNR_A05 = pd.read_csv('../data_ADNR/with0.5anomaly/allocatedJobs.csv')
df_ADNR_A05_2 = pd.read_csv('../experiments/experiment_results_NR/0.5/allocatedJobs.csv')
df_ADNR_A05 = pd.concat([df_ADNR_A05,df_ADNR_A05_2], ignore_index=True)

df_ADNR_M05 = pd.read_csv('../data_ADNR/with0.5anomaly/destroyedJobs.csv')
df_ADNR_M05_2 = pd.read_csv('../experiments/experiment_results_NR/0.5/destroyedJobs.csv')
df_ADNR_M05 = pd.concat([df_ADNR_M05,df_ADNR_M05_2], ignore_index=True)

In [44]:
df_ADNR_A05['job ressource requirement'] = df_ADNR_A05['job ressource requirement'].apply(convert_to_list)
df_ADNR_M05['job ressource requirement'] = df_ADNR_M05['job ressource requirement'].apply(convert_to_list)

df_ADNR_A05['high_resource'] = df_ADNR_A05.apply(has_high_resource, axis=1)
df_ADNR_M05['high_resource'] = df_ADNR_M05.apply(has_high_resource, axis=1)

df_ADNR_A05['high_length'] = df_ADNR_A05.apply(has_high_length, axis=1)
df_ADNR_M05['high_length'] = df_ADNR_M05.apply(has_high_length, axis=1)

df_ADNR_A05['high_length_or_resource'] = df_ADNR_A05['high_length'] | df_ADNR_A05['high_resource']
df_ADNR_M05['high_length_or_resource'] = df_ADNR_M05['high_length'] | df_ADNR_M05['high_resource']

allocated_high_length_or_resource = df_ADNR_A05['high_length_or_resource'].sum()
destroyed_high_length_or_resource = df_ADNR_M05['high_length_or_resource'].sum()

# Calculate the total number of jobs that satisfy either condition
total_high_length_or_resource = allocated_high_length_or_resource + destroyed_high_length_or_resource

# Calculate the proportion of these jobs that were destroyed
proportion_destroyed_length_or_resource = float(destroyed_high_length_or_resource) / total_high_length_or_resource

TN = len(df_ADNR_A05[(df_ADNR_A05['high_resource'] == False) & (df_ADNR_A05['high_length'] == False)])

# False negatives (anomalous jobs incorrectly identified as non-anomalies)
FN = len(df_ADNR_A05[(df_ADNR_A05['high_resource'] == True) | (df_ADNR_A05['high_length'] == True)])

TP = len(df_ADNR_M05[(df_ADNR_M05['high_resource'] == True) | (df_ADNR_M05['high_length'] == True)])

# False positives (non-anomalous jobs incorrectly identified as anomalies)
FP = len(df_ADNR_M05[(df_ADNR_M05['high_resource'] == False) & (df_ADNR_M05['high_length'] == False)])

# Recall calculation
recall = float(TP) / (TP + FN)
recall

# Precision calculation
precision = float(TP) / (TP + FP)
precision

f1_score = 2 * (precision * recall) / (precision+recall)
f1_score

print(proportion_destroyed_length_or_resource, precision, recall, f1_score)

(0.8969326062962292, 0.9189127609660597, 0.8969326062962292, 0.9077896527002017)


In [45]:
df_AD_A01 = pd.read_csv('../data_AD/anomalyrate0.1/allocatedJobs.csv')
df_AD_A01_2 = pd.read_csv('../experiments/experiment_results_AD/0.1/allocatedJobs.csv')
df_AD_A01 = pd.concat([df_AD_A01,df_AD_A01_2], ignore_index=True)

df_AD_M01 = pd.read_csv('../data_AD/anomalyrate0.1/destroyedJobs.csv')
df_AD_M01_2 = pd.read_csv('../experiments/experiment_results_AD/0.1/destroyedJobs.csv')
df_AD_M01 = pd.concat([df_AD_M01,df_AD_M01_2], ignore_index=True)

In [46]:
df_AD_A01['job ressource requirement'] = df_AD_A01['job ressource requirement'].apply(convert_to_list)
df_AD_M01['job ressource requirement'] = df_AD_M01['job ressource requirement'].apply(convert_to_list)

df_AD_A01['high_resource'] = df_AD_A01.apply(has_high_resource, axis=1)
df_AD_M01['high_resource'] = df_AD_M01.apply(has_high_resource, axis=1)

df_AD_A01['high_length'] = df_AD_A01.apply(has_high_length, axis=1)
df_AD_M01['high_length'] = df_AD_M01.apply(has_high_length, axis=1)

df_AD_A01['high_length_or_resource'] = df_AD_A01['high_length'] | df_AD_A01['high_resource']
df_AD_M01['high_length_or_resource'] = df_AD_M01['high_length'] | df_AD_M01['high_resource']

allocated_high_length_or_resource = df_AD_A01['high_length_or_resource'].sum()
destroyed_high_length_or_resource = df_AD_M01['high_length_or_resource'].sum()

# Calculate the total number of jobs that satisfy either condition
total_high_length_or_resource = allocated_high_length_or_resource + destroyed_high_length_or_resource

# Calculate the proportion of these jobs that were destroyed
proportion_destroyed_length_or_resource = float(destroyed_high_length_or_resource) / total_high_length_or_resource

TN = len(df_AD_A01[(df_AD_A01['high_resource'] == False) & (df_AD_A01['high_length'] == False)])

# False negatives (anomalous jobs incorrectly identified as non-anomalies)
FN = len(df_AD_A01[(df_AD_A01['high_resource'] == True) | (df_AD_A01['high_length'] == True)])

TP = len(df_AD_M01[(df_AD_M01['high_resource'] == True) | (df_AD_M01['high_length'] == True)])

# False positives (non-anomalous jobs incorrectly identified as anomalies)
FP = len(df_AD_M01[(df_AD_M01['high_resource'] == False) & (df_AD_M01['high_length'] == False)])

# Recall calculation
recall = float(TP) / (TP + FN)
recall

# Precision calculation
precision = float(TP) / (TP + FP)
precision

f1_score = 2 * (precision * recall) / (precision+recall)
f1_score

print(proportion_destroyed_length_or_resource, precision, recall, f1_score)

(0.7930298719772404, 0.3604992024830797, 0.7930298719772404, 0.4956730484262936)


In [47]:
df_AD_A03 = pd.read_csv('../data_AD/anomalyrate0.3/allocatedJobs.csv')
df_AD_A03_2 = pd.read_csv('../experiments/experiment_results_AD/0.3/allocatedJobs.csv')
df_AD_A03 = pd.concat([df_AD_A03,df_AD_A03_2], ignore_index=True)

df_AD_M03 = pd.read_csv('../data_AD/anomalyrate0.3/destroyedJobs.csv')
df_AD_M03_2 = pd.read_csv('../experiments/experiment_results_AD/0.3/destroyedJobs.csv')
df_AD_M03 = pd.concat([df_AD_M03,df_AD_M03_2], ignore_index=True)

In [48]:
df_AD_A03['job ressource requirement'] = df_AD_A03['job ressource requirement'].apply(convert_to_list)
df_AD_M03['job ressource requirement'] = df_AD_M03['job ressource requirement'].apply(convert_to_list)

df_AD_A03['high_resource'] = df_AD_A03.apply(has_high_resource, axis=1)
df_AD_M03['high_resource'] = df_AD_M03.apply(has_high_resource, axis=1)

df_AD_A03['high_length'] = df_AD_A03.apply(has_high_length, axis=1)
df_AD_M03['high_length'] = df_AD_M03.apply(has_high_length, axis=1)

df_AD_A03['high_length_or_resource'] = df_AD_A03['high_length'] | df_AD_A03['high_resource']
df_AD_M03['high_length_or_resource'] = df_AD_M03['high_length'] | df_AD_M03['high_resource']

allocated_high_length_or_resource = df_AD_A03['high_length_or_resource'].sum()
destroyed_high_length_or_resource = df_AD_M03['high_length_or_resource'].sum()

# Calculate the total number of jobs that satisfy either condition
total_high_length_or_resource = allocated_high_length_or_resource + destroyed_high_length_or_resource

# Calculate the proportion of these jobs that were destroyed
proportion_destroyed_length_or_resource = float(destroyed_high_length_or_resource) / total_high_length_or_resource

TN = len(df_AD_A03[(df_AD_A03['high_resource'] == False) & (df_AD_A03['high_length'] == False)])

# False negatives (anomalous jobs incorrectly identified as non-anomalies)
FN = len(df_AD_A03[(df_AD_A03['high_resource'] == True) | (df_AD_A03['high_length'] == True)])

TP = len(df_AD_M03[(df_AD_M03['high_resource'] == True) | (df_AD_M03['high_length'] == True)])

# False positives (non-anomalous jobs incorrectly identified as anomalies)
FP = len(df_AD_M03[(df_AD_M03['high_resource'] == False) & (df_AD_M03['high_length'] == False)])

# Recall calculation
recall = float(TP) / (TP + FN)
recall

# Precision calculation
precision = float(TP) / (TP + FP)
precision

f1_score = 2 * (precision * recall) / (precision+recall)
f1_score

print(proportion_destroyed_length_or_resource, precision, recall, f1_score)

(0.7957710404492676, 0.7958540866182818, 0.7957710404492676, 0.7958125613672262)


In [49]:
df_AD_A05 = pd.read_csv('../data_AD/anomalyrate0.5/allocatedJobs.csv')
df_AD_A05_2 = pd.read_csv('../experiments/experiment_results_AD/0.5/allocatedJobs.csv')
df_AD_A05 = pd.concat([df_AD_A05,df_AD_A05_2], ignore_index=True)

df_AD_M05 = pd.read_csv('../data_AD/anomalyrate0.5/destroyedJobs.csv')
df_AD_M05_2 = pd.read_csv('../experiments/experiment_results_AD/0.5/destroyedJobs.csv')
df_AD_M05 = pd.concat([df_AD_M05,df_AD_M05_2], ignore_index=True)
#df_AD_A05.head()

In [50]:
df_AD_A05['job ressource requirement'] = df_AD_A05['job ressource requirement'].apply(convert_to_list)
df_AD_M05['job ressource requirement'] = df_AD_M05['job ressource requirement'].apply(convert_to_list)

df_AD_A05['high_resource'] = df_AD_A05.apply(has_high_resource, axis=1)
df_AD_M05['high_resource'] = df_AD_M05.apply(has_high_resource, axis=1)

df_AD_A05['high_length'] = df_AD_A05.apply(has_high_length, axis=1)
df_AD_M05['high_length'] = df_AD_M05.apply(has_high_length, axis=1)

df_AD_A05['high_length_or_resource'] = df_AD_A05['high_length'] | df_AD_A05['high_resource']
df_AD_M05['high_length_or_resource'] = df_AD_M05['high_length'] | df_AD_M05['high_resource']

allocated_high_length_or_resource = df_AD_A05['high_length_or_resource'].sum()
destroyed_high_length_or_resource = df_AD_M05['high_length_or_resource'].sum()

# Calculate the total number of jobs that satisfy either condition
total_high_length_or_resource = allocated_high_length_or_resource + destroyed_high_length_or_resource

# Calculate the proportion of these jobs that were destroyed
proportion_destroyed_length_or_resource = float(destroyed_high_length_or_resource) / total_high_length_or_resource

TN = len(df_AD_A05[(df_AD_A05['high_resource'] == False) & (df_AD_A05['high_length'] == False)])

# False negatives (anomalous jobs incorrectly identified as non-anomalies)
FN = len(df_AD_A05[(df_AD_A05['high_resource'] == True) | (df_AD_A05['high_length'] == True)])

TP = len(df_AD_M05[(df_AD_M05['high_resource'] == True) | (df_AD_M05['high_length'] == True)])

# False positives (non-anomalous jobs incorrectly identified as anomalies)
FP = len(df_AD_M05[(df_AD_M05['high_resource'] == False) & (df_AD_M05['high_length'] == False)])

# Recall calculation
recall = float(TP) / (TP + FN)
recall

# Precision calculation
precision = float(TP) / (TP + FP)
precision

f1_score = 2 * (precision * recall) / (precision+recall)
f1_score

print(proportion_destroyed_length_or_resource, precision, recall, f1_score)

(0.790907033777544, 0.9429238060517682, 0.790907033777544, 0.8602512464203872)


In [53]:
df_ECO_A01M1 = pd.read_csv('../data_ECO/anomalyrate0.1/allocatedJobsM1.csv')
df_ECO_A01M1_2 = pd.read_csv('../experiment_results_ECO/0.1/allocatedJobsM1.csv')
df_ECO_A01M2 = pd.read_csv('../data_ECO/anomalyrate0.1/allocatedJobsM2.csv')
df_ECO_A01M2_2 = pd.read_csv('../experiment_results_ECO/0.1/allocatedJobsM2.csv')
df_ECO_A01 = pd.concat([df_ECO_A01M1,df_ECO_A01M1_2, df_ECO_A01M2, df_ECO_A01M2_2], ignore_index=True)

df_ECO_M01M3 = pd.read_csv('../data_ECO/anomalyrate0.1/allocatedJobsM3.csv')
df_ECO_M01M3_2 = pd.read_csv('../experiment_results_ECO/0.1/allocatedJobsM3.csv')
df_ECO_M01M3 = pd.concat([df_ECO_M01M3,df_ECO_M01M3_2], ignore_index=True)

In [54]:
df_ECO_A01['job ressource requirement'] = df_ECO_A01['job ressource requirement'].apply(convert_to_list)
df_ECO_M01M3['job ressource requirement'] = df_ECO_M01M3['job ressource requirement'].apply(convert_to_list)

df_ECO_A01['high_resource'] = df_ECO_A01.apply(has_high_resource, axis=1)
df_ECO_M01M3['high_resource'] = df_ECO_M01M3.apply(has_high_resource, axis=1)

df_ECO_A01['high_length'] = df_ECO_A01.apply(has_high_length, axis=1)
df_ECO_M01M3['high_length'] = df_ECO_M01M3.apply(has_high_length, axis=1)

df_ECO_A01['high_length_or_resource'] = df_ECO_A01['high_length'] | df_ECO_A01['high_resource']
df_ECO_M01M3['high_length_or_resource'] = df_ECO_M01M3['high_length'] | df_ECO_M01M3['high_resource']

allocated_high_length_or_resource = df_ECO_A01['high_length_or_resource'].sum()
destroyed_high_length_or_resource = df_ECO_M01M3['high_length_or_resource'].sum()

# Calculate the total number of jobs that satisfy either condition
total_high_length_or_resource = allocated_high_length_or_resource + destroyed_high_length_or_resource

# Calculate the proportion of these jobs that were destroyed
proportion_destroyed_length_or_resource = float(destroyed_high_length_or_resource) / total_high_length_or_resource

TN = len(df_ECO_A01[(df_ECO_A01['high_resource'] == False) & (df_ECO_A01['high_length'] == False)])

# False negatives (anomalous jobs incorrectly identified as non-anomalies)
FN = len(df_ECO_A01[(df_ECO_A01['high_resource'] == True) | (df_ECO_A01['high_length'] == True)])

TP = len(df_ECO_M01M3[(df_ECO_M01M3['high_resource'] == True) | (df_ECO_M01M3['high_length'] == True)])

# False positives (non-anomalous jobs incorrectly identified as anomalies)
FP = len(df_ECO_M01M3[(df_ECO_M01M3['high_resource'] == False) & (df_ECO_M01M3['high_length'] == False)])

# Recall calculation
recall = float(TP) / (TP + FN)
recall

# Precision calculation
precision = float(TP) / (TP + FP)
precision

f1_score = 2 * (precision * recall) / (precision+recall)
f1_score

print(proportion_destroyed_length_or_resource, precision, recall, f1_score)

(0.9985550361240969, 0.3053887545816112, 0.9985550361240969, 0.4677310188318669)


In [55]:
df_ECO_A03M1 = pd.read_csv('../data_ECO/anomalyrate0.3/allocatedJobsM1.csv')
df_ECO_A03M1_2 = pd.read_csv('../experiment_results_ECO/0.3/allocatedJobsM1.csv')
df_ECO_A03M2 = pd.read_csv('../data_ECO/anomalyrate0.3/allocatedJobsM2.csv')
df_ECO_A03M2_2 = pd.read_csv('../experiment_results_ECO/0.3/allocatedJobsM2.csv')
df_ECO_A03 = pd.concat([df_ECO_A03M1,df_ECO_A03M1_2, df_ECO_A03M2, df_ECO_A03M2_2], ignore_index=True)

df_ECO_M03M3 = pd.read_csv('../data_ECO/anomalyrate0.3/allocatedJobsM3.csv')
df_ECO_M03M3_2 = pd.read_csv('../experiment_results_ECO/0.3/allocatedJobsM3.csv')
df_ECO_M03M3 = pd.concat([df_ECO_M03M3,df_ECO_M03M3_2], ignore_index=True)

In [62]:
print(type(df_ECO_A03['job ressource requirement'].iloc[0]))

<type 'list'>


In [65]:
df_ECO_A03['job ressource requirement'] = df_ECO_A03['job ressource requirement'].apply(convert_to_list)
df_ECO_M03M3['job ressource requirement'] = df_ECO_M03M3['job ressource requirement'].apply(convert_to_list)

df_ECO_A03['high_resource'] = df_ECO_A03.apply(has_high_resource, axis=1)
df_ECO_M03M3['high_resource'] = df_ECO_M03M3.apply(has_high_resource, axis=1)

df_ECO_A03['high_length'] = df_ECO_A03.apply(has_high_length, axis=1)
df_ECO_M03M3['high_length'] = df_ECO_M03M3.apply(has_high_length, axis=1)

df_ECO_A03['high_length_or_resource'] = df_ECO_A03['high_length'] | df_ECO_A03['high_resource']
df_ECO_M03M3['high_length_or_resource'] = df_ECO_M03M3['high_length'] | df_ECO_M03M3['high_resource']

allocated_high_length_or_resource = df_ECO_A03['high_length_or_resource'].sum()
destroyed_high_length_or_resource = df_ECO_M03M3['high_length_or_resource'].sum()

# Calculate the total number of jobs that satisfy either condition
total_high_length_or_resource = allocated_high_length_or_resource + destroyed_high_length_or_resource

# Calculate the proportion of these jobs that were destroyed
proportion_destroyed_length_or_resource = float(destroyed_high_length_or_resource) / total_high_length_or_resource

TN = len(df_ECO_A03[(df_ECO_A03['high_resource'] == False) & (df_ECO_A03['high_length'] == False)])

# False negatives (anomalous jobs incorrectly identified as non-anomalies)
FN = len(df_ECO_A03[(df_ECO_A03['high_resource'] == True) | (df_ECO_A03['high_length'] == True)])

TP = len(df_ECO_M03M3[(df_ECO_M03M3['high_resource'] == True) | (df_ECO_M03M3['high_length'] == True)])

# False positives (non-anomalous jobs incorrectly identified as anomalies)
FP = len(df_ECO_M03M3[(df_ECO_M03M3['high_resource'] == False) & (df_ECO_M03M3['high_length'] == False)])

# Recall calculation
recall = float(TP) / (TP + FN)
recall

# Precision calculation
precision = float(TP) / (TP + FP)
precision

f1_score = 2 * (precision * recall) / (precision+recall)
f1_score

print(proportion_destroyed_length_or_resource, precision, recall, f1_score)

(0.9847071988064156, 0.732824427480916, 0.9847071988064156, 0.8402960133683457)


In [60]:
df_ECO_A05M1 = pd.read_csv('../data_ECO/anomalyrate0.5/allocatedJobsM1.csv')
df_ECO_A05M1_2 = pd.read_csv('../experiment_results_ECO/0.5/allocatedJobsM1.csv')
df_ECO_A05M2 = pd.read_csv('../data_ECO/anomalyrate0.5/allocatedJobsM2.csv')
df_ECO_A05M2_2 = pd.read_csv('../experiment_results_ECO/0.5/allocatedJobsM2.csv')
df_ECO_A05 = pd.concat([df_ECO_A05M1,df_ECO_A05M1_2, df_ECO_A05M2, df_ECO_A05M2_2], ignore_index=True)

df_ECO_M05M3 = pd.read_csv('../data_ECO/anomalyrate0.5/allocatedJobsM3.csv')
df_ECO_M05M3_2 = pd.read_csv('../experiment_results_ECO/0.5/allocatedJobsM3.csv')
df_ECO_M05M3 = pd.concat([df_ECO_M05M3,df_ECO_M05M3_2], ignore_index=True)

In [61]:
df_ECO_A05['job ressource requirement'] = df_ECO_A05['job ressource requirement'].apply(convert_to_list)
df_ECO_M05M3['job ressource requirement'] = df_ECO_M05M3['job ressource requirement'].apply(convert_to_list)

df_ECO_A05['high_resource'] = df_ECO_A05.apply(has_high_resource, axis=1)
df_ECO_M05M3['high_resource'] = df_ECO_M05M3.apply(has_high_resource, axis=1)

df_ECO_A05['high_length'] = df_ECO_A05.apply(has_high_length, axis=1)
df_ECO_M05M3['high_length'] = df_ECO_M05M3.apply(has_high_length, axis=1)

df_ECO_A05['high_length_or_resource'] = df_ECO_A05['high_length'] | df_ECO_A05['high_resource']
df_ECO_M05M3['high_length_or_resource'] = df_ECO_M05M3['high_length'] | df_ECO_M05M3['high_resource']

allocated_high_length_or_resource = df_ECO_A05['high_length_or_resource'].sum()
destroyed_high_length_or_resource = df_ECO_M05M3['high_length_or_resource'].sum()

# Calculate the total number of jobs that satisfy either condition
total_high_length_or_resource = allocated_high_length_or_resource + destroyed_high_length_or_resource

# Calculate the proportion of these jobs that were destroyed
proportion_destroyed_length_or_resource = float(destroyed_high_length_or_resource) / total_high_length_or_resource

TN = len(df_ECO_A05[(df_ECO_A05['high_resource'] == False) & (df_ECO_A05['high_length'] == False)])

# False negatives (anomalous jobs incorrectly identified as non-anomalies)
FN = len(df_ECO_A05[(df_ECO_A05['high_resource'] == True) | (df_ECO_A05['high_length'] == True)])

TP = len(df_ECO_M05M3[(df_ECO_M05M3['high_resource'] == True) | (df_ECO_M05M3['high_length'] == True)])

# False positives (non-anomalous jobs incorrectly identified as anomalies)
FP = len(df_ECO_M05M3[(df_ECO_M05M3['high_resource'] == False) & (df_ECO_M05M3['high_length'] == False)])

# Recall calculation
recall = float(TP) / (TP + FN)
recall

# Precision calculation
precision = float(TP) / (TP + FP)
precision

f1_score = 2 * (precision * recall) / (precision+recall)
f1_score

print(proportion_destroyed_length_or_resource, precision, recall, f1_score)

(0.9521979985704074, 0.8415192672141504, 0.9521979985704074, 0.8934439973172367)
